# Pathology prediction (Consolidation)


@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the Consolidation pathology using the CSV embeddings file

## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples : 0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value.

The model is based on binary classification for each target chest pathology.

Final sample size for the Consolidation pathology is : (N =  4465)


#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.CONSOLIDATION,
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="Consolidation", experiment_name="CP_Consolidation", filepath="./results/consolidation")

# Model training
evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='stratifiedkfold',
    outer_fold=5,
    outer_strategy='stratifiedkfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps
)

2024-09-26 15:25:14,862	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


(run_fold pid=469104) Outer fold 2
(run_fold pid=469104) Configuring PyCaret for outer fold 2
(run_fold pid=469091) Outer fold 1


Processing:  75%|███████▌  | 3/4 [02:47<01:02, 62.04s/it] [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(run_fold pid=469104)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=469104) Fold                                                          
(run_fold pid=469104) 0       0.8986  0.8979  0.9699  0.9111  0.9396  0.6263  0.6383
(run_fold pid=469104) 1       0.8899  0.9118  0.9720  0.9004  0.9349  0.5823  0.6006
(run_fold pid=469104) 2       0.9335  0.9586  0.9763  0.9439  0.9598  0.7663  0.7700
(run_fold pid=469104) 3       0.9037  0.9242  0.9763  0.9116  0.9429  0.6379  0.6535
(run_fold pid=469104) 4       0.8914  0.8882  0.9591  0.9119  0.9349  0.6098  0.6169
(run_fold pid=469104) Mean    0.9034  0.9161  0.9707  0.9158  0.9424  0.6445  0.6559
(run_fold pid=469104) Std     0.0158  0.0245  0.0064  0.0147  0.0092  0.0637  0.0599
(run_fold pid=469091) Configuring PyCaret for outer fold 1
(run_fold pid=469104) Tuning hyperparameters for model xgboost with custom grid using grid search


                                                         
Processing:  75%|███████▌  | 3/4 [02:48<01:02, 62.39s/it]


...

(run_fold pid=469091) 0       0.9038  0.9265  0.9720  0.9150  0.9426  0.6470  0.6585
(run_fold pid=469091) 1       0.8899  0.9060  0.9763  0.8972  0.9351  0.5752  0.5985
(run_fold pid=469091) 2       0.8967  0.9256  0.9720  0.9076  0.9387  0.6116  0.6265
(run_fold pid=469091) 3       0.9194  0.9282  0.9785  0.9267  0.9519  0.7057  0.7157
(run_fold pid=469091) 4       0.8879  0.8897  0.9591  0.9082  0.9329  0.5940  0.6023
(run_fold pid=469091)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=469091) Fold                                                          
(run_fold pid=469091) Mean    0.8995  0.9152  0.9716  0.9109  0.9403  0.6267  0.6403
(run_fold pid=469091) Std     0.0114  0.0151  0.0068  0.0097  0.0067  0.0460  0.0433
(run_fold pid=469091) Tuning hyperparameters for model xgboost with custom grid using grid search


(run_fold pid=469104) Transformation Pipeline and Model Successfully Saved
(run_fold pid=469104)                        Model  Accuracy     AUC  ...     F1   Kappa     MCC
(run_fold pid=469104) 0  Extreme Gradient Boosting    0.9104  0.9406  ...  0.946  0.6835  0.6884
(run_fold pid=469104) 
(run_fold pid=469104) [1 rows x 8 columns]
(run_fold pid=469104) Outer fold 5
(run_fold pid=469104) Configuring PyCaret for outer fold 5


Processing:  75%|███████▌  | 3/4 [02:29<00:55, 55.17s/it]


(run_fold pid=469104)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=469104) Fold                                                          
(run_fold pid=469104) 0       0.8934  0.8945  0.9656  0.9089  0.9364  0.6085  0.6193
(run_fold pid=469104) 1       0.9091  0.9206  0.9742  0.9189  0.9457  0.6676  0.6786
(run_fold pid=469104) 2       0.9054  0.9067  0.9720  0.9168  0.9436  0.6517  0.6626
(run_fold pid=469104) 3       0.9107  0.9308  0.9806  0.9157  0.9470  0.6642  0.6805
(run_fold pid=469104) 4       0.8914  0.8867  0.9720  0.9020  0.9357  0.5905  0.6079
(run_fold pid=469104) Mean    0.9020  0.9079  0.9729  0.9125  0.9417  0.6365  0.6498
(run_fold pid=469104) Std     0.0081  0.0162  0.0048  0.0062  0.0047  0.0312  0.0304
(run_fold pid=469104) Tuning hyperparameters for model xgboost with custom grid using grid search


(run_fold pid=469091) Transformation Pipeline and Model Successfully Saved
(run_fold pid=469091)                        Model  Accuracy     AUC  ...     F1   Kappa     MCC
(run_fold pid=469091) 0  Extreme Gradient Boosting    0.9071  0.9376  ...  0.945  0.6481  0.6679
(run_fold pid=469091) 
(run_fold pid=469091) [1 rows x 8 columns]
(run_fold pid=469104) Transformation Pipeline and Model Successfully Saved
(run_fold pid=469104)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=469104) 0  Extreme Gradient Boosting    0.9093  0.9518  ...  0.9462  0.6602  0.6771
(run_fold pid=469104) 
(run_fold pid=469104) [1 rows x 8 columns]
Final metrics table:
     Metric     Mean   Std Dev
0  Accuracy  0.89718  0.006071
1       AUC  0.91178  0.015454
2    Recall  0.97072  0.009087
3     Prec.  0.90902  0.004655
4        F1  0.93884  0.003827
5     Kappa  0.61870  0.019738
6       MCC  0.63286  0.021338
